### Student Information
Name:Dongil (Jeff) Jang

Student ID:108065429

GitHub ID:justdoit0430

---

### Instructions

1. First: do the **take home** exercises in the [DM19-Lab1-Master Repo](https://github.com/EvaArevalo/DM19-Lab1-Master). You may need to copy some cells from the Lab notebook to this notebook. __This part is worth 20% of your grade.__


2. Second: follow the same process from the [DM19-Lab1-Master Repo](https://github.com/EvaArevalo/DM19-Lab1-Master) on **the new dataset**. You don't need to explain all details as we did (some **minimal comments** explaining your code are useful though).  __This part is worth 30% of your grade.__
    - Download the [the new dataset](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences#). The dataset contains a `sentence` and `score` label. Read the specificiations of the dataset for details. 
    - You are allowed to use and modify the `helper` functions in the folder of the first lab session (notice they may need modification) or create your own.


3. Third: please attempt the following tasks on **the new dataset**. __This part is worth 30% of your grade.__
    - Generate meaningful **new data visualizations**. Refer to online resources and the Data Mining textbook for inspiration and ideas. 
    - Generate **TF-IDF features** from the tokens of each text. This will generating a document matrix, however, the weights will be computed differently (using the TF-IDF value of each word per document as opposed to the word frequency). Refer to this Sciki-learn [guide](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) .
    - Implement a simple **Naive Bayes classifier** that automatically classifies the records into their categories. Use both the TF-IDF features and word frequency features to build two seperate classifiers. Comment on the differences.  Refer to this [article](https://hub.packtpub.com/implementing-3-naive-bayes-classifiers-in-scikit-learn/).


4. Fourth: In the lab, we applied each step really quickly just to illustrate how to work with your dataset. There are somethings that are not ideal or the most efficient/meaningful. Each dataset can be habdled differently as well. What are those inefficent parts you noticed? How can you improve the Data preprocessing for these specific datasets? __This part is worth 10% of your grade.__


5. Fifth: It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**. __This part is worth 10% of your grade.__


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/EvaArevalo/DM19-Lab1-Master/blob/master/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb). Make sure to commit and save your changes to your repository __BEFORE the deadline (Oct. 29th 11:59 pm, Tuesday)__. 

In [17]:
# import library
import pandas as pd
import numpy as np
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import plotly as py
import math
import matplotlib.pyplot as plt #추가
import os
import glob
import helpers.data_mining_helpers as dmh

%matplotlib inline

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
#importing each txt files and see they are all in same format in order to be combined into one.
#At this stage, I was thinking that making a new category column for each 
#import pandas as pd
#amazon=pd.read_csv("./amazon.txt",header=None, sep='\n')
#amazon = amazon[0].str.split(',', expand=True)
#imdb=pd.read_csv("./imdb.txt",header=None, sep='\n')
#imdb= imdb[0].str.split(',', expand=True)
#yelp=pd.read_csv("./yelp.txt",header=None, sep='\n')
#yelp= yelp[0].str.split(',', expand=True)

#ignore it

In [5]:
def opencsv(path):
    with open(path) as file:
        data = file.read()
    return data
imdb = opencsv('./imdb.txt')
amazon = opencsv('./amazon.txt')
yelp = opencsv('./yelp.txt')

In [6]:
datasets = [imdb, amazon, yelp] #이 부분 이름 바뀐건 확인할것

In [7]:
#need to combine three datas into one.

combined_dataset = []
for dataset in datasets:
    combined_dataset.extend(dataset.split('\n'))
dataset = [sample.split('\t') for sample in combined_dataset]


In [8]:
df = pd.DataFrame(data=dataset, columns=['Reviews', 'Score'])

df = df[df["Score"].notnull()] #To elimiate all the null data


In [9]:
len(df)

3000

In [10]:
df.head()

,Reviews,Score
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [11]:
df[-10:]

,Reviews,Score
2992,The refried beans that came with my meal were ...,0
2993,Spend your money and time some place else.,0
2994,A lady at the table next to us found a live gr...,0
2995,the presentation of the food was awful.,0
2996,I can't tell you how disappointed I was.,0
2997,I think food should have flavor and texture an...,0
2998,Appetite instantly gone.,0
2999,Overall I was not impressed and would not go b...,0
3000,"The whole experience was underwhelming, and I ...",0
3001,"Then, as if I hadn't wasted enough of my life ...",0


In [12]:
# using loc (by position)
df.iloc[::10, 0:2][0:10]

,Reviews,Score
0,"A very, very, very slow-moving, aimless movie ...",0
10,And those baby owls were adorable.,1
20,I can think of no other film where something v...,1
30,"There are massive levels, massive unlockable c...",1
40,The very idea of it was lame - take a minor ch...,0
50,I'm glad the film didn't go for the most obvio...,1
60,I sat riveted to the TV screen.,1
70,"And, quite honestly, often its not very good.",0
80,I'm glad this pretentious piece of s*** didn't...,0
90,Definitely worth checking out.,1


In [13]:
# using loc (by label)
df.loc[::10, 'Reviews'][0:10]

0     A very, very, very slow-moving, aimless movie ...
10                 And those baby owls were adorable.  
20    I can think of no other film where something v...
30    There are massive levels, massive unlockable c...
40    The very idea of it was lame - take a minor ch...
50    I'm glad the film didn't go for the most obvio...
60                    I sat riveted to the TV screen.  
70      And, quite honestly, often its not very good.  
80    I'm glad this pretentious piece of s*** didn't...
90                     Definitely worth checking out.  
Name: Reviews, dtype: object

In [14]:
df.isnull()

,Reviews,Score
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
2997,False,False
2998,False,False
2999,False,False
3000,False,False


In [15]:
df.isnull().apply(lambda x: dmh.check_missing_values(x))

Reviews    (The amoung of missing records is: , 0)
Score      (The amoung of missing records is: , 0)
dtype: object

In [27]:
#just try to practice the dummy process that was done from the Lab class.
dummy_series = pd.Series(["dummy_record", None], index=["Reviews", "Score"])

In [28]:
dummy_series

Reviews    dummy_record
Score              None
dtype: object

In [29]:
result_with_series = df.append(dummy_series, ignore_index=True)

In [30]:
# check if the records was commited into result
len(result_with_series)

3001

In [31]:
result_with_series.isnull().apply(lambda x: dmh.check_missing_values(x))

Reviews    (The amoung of missing records is: , 0)
Score      (The amoung of missing records is: , 1)
dtype: object

In [33]:
df.dropna(inplace=True)

In [34]:
df.isnull().apply(lambda x: dmh.check_missing_values(x))

Reviews    (The amoung of missing records is: , 0)
Score      (The amoung of missing records is: , 0)
dtype: object

In [35]:
len(df)

3000

Dealing the duplicate data

In [36]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2997    False
2998    False
2999    False
3000    False
3001    False
Length: 3000, dtype: bool

In [41]:
sum(df.duplicated()) 

17

In [42]:
df.drop_duplicates(keep=False, inplace=True) 

In [43]:
sum(df.duplicated()) 

0

In [45]:
df.shape

(2966, 2)

5. Data Preprocessing

Sampling

In [46]:
df_sample = df.sample(n=1000) 

In [47]:
len(df_sample)

1000

In [48]:
df_sample[0:4]

,Reviews,Score
2264,The restaurant atmosphere was exquisite.,1
2040,My first visit to Hiro was a delight!,1
340,But it wasn't anything at all just one big yaw...,0
1050,Disappointed with battery.,0
